## Demo - BandMaths

In [1]:
import esa_snappy.snapista

from snapista import Graph
from snapista import Operator
from snapista import read_file
from snapista import TargetBand
from snapista import TargetBandDescriptors

#import os 
#os.environ['PATH'] = '/srv/conda/envs/env_snap/snap/bin:' +  os.environ['PATH']

In [2]:

try:
    g = Graph()
except Exception:
    # running in vs container
    import os 
    os.environ['PATH'] = '/srv/conda/envs/env_snap/snap/bin:' +  os.environ['PATH']
    
    g = Graph()

In [3]:
g.add_node(operator=Operator("Read", file='/data/S2B_MSIL2A_20210608T112119_N0300_R037_T29SMC_20210608T131325.SAFE/MTD_MSIL2A.xml'), node_id="Read")


In [4]:
g.add_node(operator=Operator("Resample", referenceBandName="B2"), node_id="Resample", source="Read")

In [5]:
band_maths = Operator('BandMaths')

cloud_class = TargetBand(name='Cloud_Class',
                              expression='if quality_cloud_confidence > 90 then 2 else (if quality_cloud_confidence > 50 AND B8 > 0.3 then 1 else 0)')
                         
band_maths.targetBandDescriptors = TargetBandDescriptors([cloud_class])

In [6]:
g.add_node(operator=band_maths, node_id="BandMaths" ,source='Resample')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x000002ACDFFF4040>
Instance TargetBandDescriptors: True
Instance Aggregators: False
Instance BinningOutputBands: False
Instance BinningVariables: False
Instance str: False


In [7]:
band_maths = Operator('BandMaths')

ndsi = TargetBand(name='NDSI',
                              expression='(B3-B11)/(B3+B11)')
                         
band_maths.targetBandDescriptors = TargetBandDescriptors([ndsi])

g.add_node(operator=band_maths, node_id="BandMaths_2" ,source='Resample')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x000002ACDFFF4040>
Instance TargetBandDescriptors: True
Instance Aggregators: False
Instance BinningOutputBands: False
Instance BinningVariables: False
Instance str: False


In [8]:
merge = Operator("BandMerge", sourceBandNames="B3,B4,B11,B12,Cloud_Class,NDSI", source=["Resample", "BandMaths", 'BandMaths_2'])

merge

Operator('BandMerge', sourceBandNames='B3,B4,B11,B12,Cloud_Class,NDSI', geographicError='1.0E-5f', source='['Resample', 'BandMaths', 'BandMaths_2']')

In [9]:
g.add_node(operator=merge, node_id="Merge", source=["Resample", "BandMaths", 'BandMaths_2'])

In [10]:
write = Operator("Write")

write

Operator('Write', file='None', formatName='BEAM-DIMAP', deleteOutputOnFailure='true', writeEntireTileRows='false', clearCacheAfterRowWrite='false')

In [11]:
g.add_node(operator=Operator("Write", file='snow.dim'), node_id="Write", source='Merge')

In [12]:
g.view()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>/data/S2B_MSIL2A_20210608T112119_N0300_R037_T29SMC_20210608T131325.SAFE/MTD_MSIL2A.xml</file>
      <formatName/>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
      <useAdvancedOptions>false</useAdvancedOptions>
    </parameters>
  </node>
  <node id="Resample">
    <operator>Resample</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandResamplings/>
      <downsamplingMethod>First</downsamplingMethod>
      <flagDownsamplingMethod>First</flagDownsamplingMethod>
      <referenceBandName>B2</referenceBandName>
      <resampleOnPyramidLevels>true</resampleOnPyramidLevels>
      <resamplingPreset/>
      <targetHeight/>
      <targetResolution/>


In [13]:
g.save_graph('snow.xml')
g.run()

Processing the graph
Executing processing graph
SEVERE: org.esa.snap.runtime.Engine: Failed to modify class loader field 'usr_paths'
done.
java.lang.NoSuchMethodException: java.lang.ClassLoader.initializePath(java.lang.String)
at java.base/java.lang.Class.getDeclaredMethod(Class.java:2848)
at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:286)
at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:265)
at org.esa.snap.runtime.Engine.<init>(Engine.java:46)
at org.esa.snap.runtime.Engine.start(Engine.java:117)
at org.esa.snap.runtime.Engine.start(Engine.java:90)
at org.esa.snap.runtime.Launcher.run(Launcher.java:51)
at org.esa.snap.runtime.Launcher.main(Launcher.java:31)
at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
at java.base/java.lang.reflect.Method.invoke(Method.java:580)
at com.exe4j.runtime.LauncherEngine.launch(LauncherEngine.java:84)
at com.exe4j.runtime.WinLauncher.main(WinLauncher.java:94)
at com.ins

PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'C:\\Users\\olafd\\AppData\\Local\\Temp\\tmp0qxcww99'